In [ ]:
from MotionGPT import *

In [ ]:
loader = FBXDataLoader("FBXDataStand", has_translate=True)

In [ ]:
loader.LoadData()

In [ ]:
len(loader.raw_data)

In [ ]:
len(loader.raw_data[0])

In [ ]:
loader.raw_data[1][0]

In [ ]:
loader.raw_data[1][30]

In [ ]:
loader.raw_data[0][0][1][0:3]

In [ ]:
loader.PrepareTrainingData(12)

In [ ]:
len(loader.all_data)

In [ ]:
frame_data = loader.raw_data[0][0]

In [ ]:
frame_data[0]

In [ ]:
mirror_frame_data = []
for i in range(len(G_MIXAMO_JOINTS)):
    joint = G_MIXAMO_JOINTS[i]
    if "Left" in joint:
        mirror_joint = joint.replace("Left", "Right")
        mirror_index = G_MIXAMO_JOINTS.index(mirror_joint)
        joint_data = [_ for _ in frame_data[mirror_index]]
        
        #mirror
        joint_data[0] = -joint_data[0]
        joint_data[-1] = -joint_data[-1]
        joint_data[-2] = -joint_data[-2]
        
        mirror_frame_data.append(joint_data)
        
    elif "Right" in joint:
        mirror_joint = joint.replace("Right", "Left")
        mirror_index = G_MIXAMO_JOINTS.index(mirror_joint)
        joint_data = [_ for _ in frame_data[mirror_index]]
        
        #mirror
        joint_data[0] = -joint_data[0]
        joint_data[-1] = -joint_data[-1]
        joint_data[-2] = -joint_data[-2]
        
        mirror_frame_data.append(joint_data)
    
    else:
        joint_data = [_ for _ in frame_data[i]]
        
        #mirror
        joint_data[0] = -joint_data[0]
        joint_data[-1] = -joint_data[-1]
        joint_data[-2] = -joint_data[-2]
        
        mirror_frame_data.append(joint_data)
    
        
        

In [ ]:
for i in range(len(frame_data)):
    print("i----------",G_MIXAMO_JOINTS[i])
    print(frame_data[i])
    print(mirror_frame_data[i])

In [ ]:
mirror_frame_data

In [ ]:
len(loader.train_data)

In [ ]:
len(loader.test_data)

In [ ]:
loader.test_data[0]

In [ ]:
len(loader.train_data[180])

In [ ]:
from VRNN import *

In [ ]:
input_dim = len(loader.train_data[0][1])
h_dim = 64
z_dim = 10
n_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #device

In [ ]:
print(device)

In [ ]:
model = VRNN(input_dim, h_dim, z_dim, n_layers, device)
model.to(device)

In [ ]:
batch_size = 16
n_epochs = 100
clip = 10
learning_rate = 1e-3
print_every = 10

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(n_epochs):
    batch_idx = 0   
    for batch_data, pad_data in loader.next_batch(batch_size):
        batch_idx += 1

        batch_data = batch_data.to(device)
        pad_data = pad_data.to(device)

        optimizer.zero_grad()

        kld_loss, nll_loss, _, _ = model(batch_data, pad_data)
        loss = kld_loss + nll_loss
        loss.backward()

        #grad norm clipping, only in pytorch version >= 1.10
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        optimizer.step()

        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(batch_data), len(loader.train_data),
                100. * batch_idx / len(loader.train_data),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

In [ ]:
model.sample(20)

-------------------------------------

In [ ]:
from MotionGPT import *

In [ ]:
maker = FBXDataMaker(12345, radian=True)

In [ ]:
maker.mc.UndoToBeginning(10000)

In [ ]:
maker.GetModelNameSpace()
print(maker.namespace)

In [ ]:
maker.LoadModel("runs/2020_08_19_18_04_29_nr.pth")

In [ ]:
maker.SampleAnim(10)

In [ ]:
#loader 
loader = FBXDataLoader("FBXDataStand", radian=True)
loader.LoadData()
loader.PrepareTrainingData(12)

In [ ]:
train_sample = torch.FloatTensor(loader.train_data[np.random.randint(2000)])

In [ ]:
train_sample = train_sample.unsqueeze(1)

In [ ]:
train_sample.shape

In [ ]:
a, b = next(iter(loader.next_batch()))

In [ ]:
a.shape, b.shape

In [ ]:
train_sample_mask = torch.LongTensor([[1]]*train_sample.size(0))

In [ ]:
train_sample_mask.shape

In [ ]:
train_sample = train_sample.to(maker.model.device)
train_sample_mask = train_sample_mask.to(maker.model.device)


In [ ]:
result = maker.model.forward(train_sample, train_sample_mask)

In [ ]:
for i in range(len(train_sample)):
    maker.GenerateMayaPose(result[3][0][i][0], i * frame_gap)

In [ ]:
result[3][0][1]

In [ ]:
maker_ori = FBXDataMaker(12346, radian=True)

In [ ]:
maker_ori.GetModelNameSpace()
print(maker_ori.namespace)

In [ ]:
maker_ori.mc.UndoToBeginning(10000)

In [ ]:
for i in range(len(train_sample)):
    maker_ori.GenerateMayaPose(train_sample[i][0], i * frame_gap)

In [ ]:
np.sqrt(3000/63)

In [ ]:
70*0.6